In [8]:
# CUDA 11.8 例子；H100 / CUDA 12.x 把 cu118 改 cu122 且版本成对
!pip uninstall -y torch torchvision torchaudio -q
!pip install --no-cache-dir -q \
  torch==2.2.2+cu118 torchvision==0.17.2+cu118 torchaudio==2.2.2 \
  --extra-index-url https://download.pytorch.org/whl/cu118            # PyTorch 官方轮子 :contentReference[oaicite:1]{index=1}
!pip install -q "vllm>=0.4.2,<=0.4.5" --no-build-isolation   # vLLM & Flash-Attention 2 :contentReference[oaicite:2]{index=2}

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.26.post1 requires torch==2.3.0, but you have torch 2.2.2+cu118 which is incompatible.
vllm 0.4.2 requires torch==2.3.0, but you have torch 2.2.2+cu118 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.17.2+cu118 requires torch==2.2.2, but you have torch 2.3.0 which is incompatible.
torchaudio 2.2.2+cu118 requires torch==2.2.2, but you have torch 2.3.0 which is incompatible.


In [ ]:
# 克隆代码
!git clone https://github.com/THUDM/LongWriter.git
%cd LongWriter

# 安装依赖
!pip install -r requirements.txt

In [6]:
#@title 📦 安装模型依赖
!pip install -q transformers>=4.43.0 accelerate sentencepiece einops

In [16]:
#@title 下载 LongWriter-GLM4-9B 到 /content
from huggingface_hub import snapshot_download

model_id = "THUDM/LongWriter-llama3.1-8b"  # 也可改成 THUDM/LongWriter-llama3.1-8b
model_path = snapshot_download(repo_id=model_id, local_dir="./longwriter", local_dir_use_symlinks=False)
print("权重已保存到：", model_path)

/venv/main/lib/python3.10/site-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/872 [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

权重已保存到： /workspace/longwriter


In [ ]:
!huggingface-cli login

In [4]:
!pip install tiktoken

In [ ]:
export VLLM_DISABLE_VISION=1

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch, textwrap, time


model_id = "THUDM/LongWriter-llama3.1-8b"
tok    = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model  = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,        # bfloat16 亦可
            device_map="auto",
            trust_remote_code=True).eval()

prompt = "请用不超过 8000 字续写『少年拿起长剑，看向破碎的天空』\n\n"
input_ids = tok(prompt, return_tensors="pt").input_ids.to(model.device)

# *可选*：流式打印
streamer = TextStreamer(tok, skip_special_tokens=True)

start = time.time()
out_ids = model.generate(
            input_ids,
            max_new_tokens=4096,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            streamer=streamer   # 若不想流式就删掉
         )
print("\n⏱️ 用时 %.1f s" % (time.time() - start))

# 如果没用 streamer，可在最后一起 decode
# print(textwrap.shorten(tok.decode(out_ids[0], skip_special_tokens=True), 800))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


请用不超过 8000 字续写『少年拿起长剑，看向破碎的天空』

少年拿起长剑，看向破碎的天空，心中涌起一股无尽的悲伤和愤怒。他知道，这一切都是因为那恶魔的出现而起。

恶魔，一个拥有无尽力量的存在，它的出现让整个世界都陷入了混乱。它的力量足以摧毁整个星球，而少年，唯一能够对抗恶魔的人，却只能孤独地站在战场上。

少年的名字叫做李昊，他是一个普通的高中生，直到那一天，他的生活被彻底改变了。他的父母在一次意外中去世，他的家人也被恶魔所伤害。从此以后，他的生活就变得充满了痛苦和绝望。

但是，李昊并没有放弃，他坚信自己能够战胜恶魔，拯救这个世界。他开始努力修炼，学习各种战斗技巧和魔法。他每天都在训练场上挥舞长剑，磨练自己的技艺。他知道，只有变得足够强大，才能保护自己珍视的一切。

终于，李昊等待的那一天到来了。他站在战场上，面对着恶魔，毫不畏惧。他举起长剑，释放出强大的力量，将恶魔击退。

然而，恶魔并没有那么容易被击败。它不断地攻击李昊，让他陷入了困境。李昊拼尽全力，终于在最后一刻将恶魔击败。

战场上充满了硝烟和血腥，李昊疲惫地倒在地上。他知道，他已经赢得了这场战斗，但他也付出了巨大的代价。他失去了很多亲人和朋友，但他也获得了更多的力量和勇气。

从此以后，李昊成为了世界的守护者。他用自己的力量保护着这个世界，让它免受恶魔的侵害。他成为了人们心中的英雄，也成为了一个传奇。

然而，李昊并没有因此而骄傲自满。他知道，他的使命还没有结束。他还要继续努力，保护这个世界，直到他的最后一刻。

李昊拿起长剑，看向破碎的天空，他知道，他的战斗还没有结束。他要继续前进，直到他实现他的梦想，拯救这个世界。

⏱️ 用时 5936.2 s


In [14]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "THUDM/LongWriter-llama3.1-8b"
pipe = pipeline(
        "text-generation",
        model=model_id,
        tokenizer=model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
)

resp = pipe(
    "### 指令：请写一篇 3000 字的赛博武侠短篇小说。\n### 回复：",
    max_new_tokens=4096,
    temperature=0.7,
    top_p=0.9
)[0]["generated_text"]
print(resp[:800], "...")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


RuntimeError: Numpy is not available

In [ ]:
#@title 启动 vLLM 服务并生成
%%bash
python - <<'PY'
from vllm import LLM, SamplingParams

prompt = "Summarize 'Romance of the Three Kingdoms' in 3000 Chinese characters."
sampler = SamplingParams(temperature=0.4, max_tokens=4096)
llm = LLM(model="THUDM/LongWriter-glm4-9b", dtype="bfloat16")

out = llm.generate(prompt, sampler)
print(out[0].outputs[0].text[:800], "...")
PY

In [ ]:
#@title GPT-4o 质量评测（需 API KEY）
!export OPENAI_API_KEY="your-default-api-key"  # 填写自己的 Key

from evaluation.eval_quality import evaluate_quality
# 传入 model_path、数据集路径等参数，脚本会自动调用 GPT-4o 进行打分
evaluate_quality(model_name_or_path="./longwriter",
                 dataset_name="THUDM/LongWriter-6k",
                 split="test",
                 output_file="quality_scores.jsonl")

In [ ]:
# 假设已克隆仓库并安装依赖，模型放在 ./longwriter
python -m pip install -q rouge-score bert-score jieba # Rouge/中文分词
export CUDA_VISIBLE_DEVICES=0

In [ ]:
python evaluation/pred.py \
  --model_path ./longwriter \
  --dataset THUDM/LongWriter-6k \
  --split test \
  --output_file gen_longwriter.jsonl \
  --max_new_tokens 4096 \
  --batch_size 1

In [ ]:
python evaluation/eval_quality.py \
  --prediction_file gen_longwriter.jsonl \
  --metrics rouge bertscore bleurt \
  --language zh

In [ ]:
export OPENAI_API_KEY="your-default-api-key"      # 需自备
python evaluation/eval_quality.py \
  --prediction_file gen_longwriter.jsonl \
  --judge gpt4o \
  --gpt_max_tokens 300

In [3]:
#@title 🐍 安装 PyTorch + TorchVision（官方轮子）
!pip uninstall -y torch torchaudio -q

!pip install --no-cache-dir -q \
  torch==2.2.2+cu118 torchvision==0.17.2+cu118 torchaudio==2.2.2 \
  --extra-index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.2/819.2 MB 174.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 178.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 342.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 321.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 160.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 174.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 189.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 155.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 314.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 186.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
#@title 📦 安装模型依赖
!pip install -q transformers>=4.43.0 accelerate sentencepiece einops
# （可选）Flash-Attention 2——节省显存/提速
# （可选）高速推理 vLLM
!pip install -q "vllm>=0.4.0,<=0.4.2"

In [ ]:
#@title ⬇️ Hugging Face snapshot_download
from huggingface_hub import snapshot_download
model_id = "THUDM/LongWriter-glm4-9b"        # 或 LongWriter-llama3.1-8b
local_dir = snapshot_download(model_id, local_dir="./longwriter", local_dir_use_symlinks=False)
print("模型保存于", local_dir)

In [ ]:
!pip install torchvision

In [ ]:
import torch, torchvision, torchvision.ops
print(torch.__version__)        # 2.3.0+cu122
print(torchvision.__version__)  # 0.18.0+cu121  (或 0.18.0)
print("NMS ok:", hasattr(torchvision.ops, "nms"))  # True

In [21]:
!export VLLM_DISABLE_VISION=1

In [26]:
!pip install "vllm<0.4"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 67.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 MB 41.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 17.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 112.4 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="./longwriter",
    dtype="bfloat16",
    max_model_len=32768,
)


params = SamplingParams(max_tokens=4096, temperature=0.4)
out = llm.generate("写 5000 字的科幻小说大纲。", params)
print(out[0].outputs[0].text[:800], "…")

TypeError: EngineArgs.__init__() got an unexpected keyword argument 'rope_scaling'

In [12]:
!nvidia-smi

Wed Apr 30 11:14:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:C7:00.0 Off |                    0 |
| N/A   31C    P0              59W / 400W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install vllm # If you are using pip.